# Predictive Analysis on Airline Delay Data using Machine Learning

#### This notebook demonstrates the use of machine learning techniques to predict flight delays. The task includes feature selection, model training, and evaluation using airline flight data.

## Objective

The objective of this task is to build a machine learning model that predicts whether a flight
will be delayed based on selected operational and temporal features.


In [1]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


## Dataset Loading

The airline dataset is loaded using Dask to handle large-scale data efficiently.

In [2]:
df = dd.read_csv(
    r"D:\bigData\airline_data\2018.csv",
    assume_missing=True,
    dtype={"CANCELLATION_CODE": "object"}
)

In [3]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429.0,EWR,DEN,1517.0,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427.0,LAS,SFO,1115.0,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426.0,SNA,DEN,1335.0,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425.0,RSW,ORD,1546.0,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424.0,ORD,ALB,630.0,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


## Target Variable

A new target variable `DELAYED` is created.
A flight is considered delayed if the departure delay exceeds 15 minutes.

In [4]:
df["DELAYED"] = df["DEP_DELAY"] > 15

## Feature Selection

The following features are selected based on their relevance to flight delays:

- Distance
- Air time
- Month
- Day of the week


In [5]:
df.columns

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27', 'DELAYED'],
      dtype='object')

## Feature Engineering from Date Column

The `FL_DATE` column was converted to datetime format.
From this column, new temporal features such as Month and Day of Week were extracted.
These features help capture seasonal and weekly patterns in flight delays.

In [6]:
df["FL_DATE"] = dd.to_datetime(df["FL_DATE"])

In [7]:
df["MONTH"] = df["FL_DATE"].dt.month

In [8]:
df["DAY_OF_WEEK"] = df["FL_DATE"].dt.dayofweek

In [9]:
features = ["DISTANCE", "AIR_TIME", "MONTH", "DAY_OF_WEEK"]

## Data Preparation

Since machine learning models require in-memory data, a sample of the dataset
is converted from Dask to Pandas.

In [10]:
sample_df = df[features + ["DELAYED"]].dropna().sample(frac=0.1)

In [11]:
sample_df.compute()

,DISTANCE,AIR_TIME,MONTH,DAY_OF_WEEK,DELAYED
423393,862.0,121.0,1,2,False
46243,792.0,139.0,1,2,False
46184,458.0,78.0,1,2,False
315811,1391.0,184.0,1,3,False
262846,594.0,97.0,1,0,True
...,...,...,...,...,...
344048,345.0,51.0,12,4,False
383554,1521.0,231.0,12,6,False
455036,508.0,84.0,12,2,False
37076,507.0,76.0,12,2,False


## Train-Test Split

The dataset is split into training and testing sets.

In [12]:
sample_df = sample_df.compute()
X = sample_df[features]
y = sample_df["DELAYED"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Model Training

A Logistic Regression model is used for binary classification.


In [13]:
model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

## Model Evaluation

The performance of the model is evaluated using accuracy and classification metrics.

In [14]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

print(classification_report(y_test, y_pred, zero_division=0))

Model Accuracy: 0.5551984743607854
              precision    recall  f1-score   support

       False       0.83      0.58      0.68    116391
        True       0.18      0.44      0.26     25189

    accuracy                           0.56    141580
   macro avg       0.51      0.51      0.47    141580
weighted avg       0.71      0.56      0.61    141580



## Model Evaluation and Interpretation

The Logistic Regression model was evaluated using accuracy, precision, recall, and F1-score.

### Evaluation Metrics

- **Accuracy (55.5%)**  
  Accuracy represents the overall correctness of predictions. However, due to class imbalance in the dataset, accuracy alone is not a reliable metric.

- **Precision (Delayed Flights – 0.18)**  
  Precision indicates how many flights predicted as delayed were actually delayed. The low precision suggests that some non-delayed flights were incorrectly classified as delayed.

- **Recall (Delayed Flights – 0.44)**  
  Recall measures how many actual delayed flights were correctly identified by the model. A recall of 44% shows that the model successfully detects a significant portion of delayed flights, which is crucial for operational decision-making.

- **F1-Score (Delayed Flights – 0.26)**  
  The F1-score balances precision and recall, indicating moderate performance in identifying delayed flights.

### Interpretation

- The dataset is highly imbalanced, with delayed flights forming a minority class.
- The model prioritizes recall over precision to ensure delayed flights are identified rather than missed.
- Compared to a baseline model that predicted no delays, this model provides meaningful predictions.
- The trade-off between accuracy and recall highlights the importance of selecting appropriate evaluation metrics for imbalanced data.


## Insights

- The dataset is highly imbalanced, with delayed flights forming a minority class.
- Accuracy alone was misleading, as earlier models failed to identify delayed flights.
- Using class-weight balancing significantly improved the model’s recall for delayed flights.
- The model successfully identified 44% of delayed flights, indicating better real-world usefulness.
- There is a trade-off between overall accuracy and the ability to detect flight delays.

## Conclusion

This predictive modeling task highlights the importance of handling class imbalance in real-world datasets.
While the overall accuracy decreased, the model became significantly more effective at identifying delayed flights,
which is the primary business objective.

By applying Logistic Regression with class weighting, the model achieved improved recall for delayed flights.
Further enhancements such as advanced models, feature engineering, or resampling techniques can improve performance further.